In [4]:
!pip install mysql-connector-python

     ---------------------------------------- 7.9/7.9 MB 540.8 kB/s eta 0:00:00


In [1]:
import mysql.connector
import pandas as pd

In [2]:
# CREATE CONNECTION
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='Password_untuk_SQL',
    database='world')

In [8]:
# WORKFRAME
# CREATE ACCES TO DATABASE
mycursor = mydb.cursor()

# QUERY
query='SELECT * FROM country;'

# EXECUTE QUERY
mycursor.execute(query)

# SAVE RESULT
result = mycursor.fetchall()

# CONVERT INTO DATAFRAME
pd.DataFrame(result, columns=mycursor.column_names)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.00,NaN,103000,78.4,828.00,793.00,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.00,1919.0,22720000,45.9,5976.00,None,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.00,1975.0,12878000,38.3,6648.00,7984.00,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.00,NaN,8000,76.1,63.20,None,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.00,1912.0,3401200,71.6,3205.00,2500.00,Shqipëria,Republic,Rexhep Mejdani,34.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,YEM,Yemen,Asia,Middle East,527968.00,1918.0,18112000,59.8,6041.00,5729.00,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.00,1918.0,10640000,72.4,17000.00,None,Jugoslavija,Federal Republic,Vojislav Koštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.00,1910.0,40377000,51.1,116729.00,129092.00,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.00,1964.0,9169000,37.2,3377.00,3922.00,Zambia,Republic,Frederick Chiluba,3162.0,ZM


In [9]:
# BUAT FUNCTION WORKFRAME
def sql(query):
    mycursor.execute(query)
    result=mycursor.fetchall()
    df=pd.DataFrame(result,columns=mycursor.column_names)
    return df

In [11]:
query_1 = 'SELECT Name FROM country;'
sql(query_1).head()

,Name
0,Aruba
1,Afghanistan
2,Angola
3,Anguilla
4,Albania


In [12]:
sql('SELECT * FROM city WHERE countrycode="idn" ')

,ID,Name,CountryCode,District,Population
0,939,Jakarta,IDN,Jakarta Raya,9604900
1,940,Surabaya,IDN,East Java,2663820
2,941,Bandung,IDN,West Java,2429000
3,942,Medan,IDN,Sumatera Utara,1843919
4,943,Palembang,IDN,Sumatera Selatan,1222764
...,...,...,...,...,...
80,1019,Batam,IDN,Riau,91871
81,1020,Padang Sidempuan,IDN,Sumatera Utara,91200
82,1021,Sawangan,IDN,West Java,91100
83,1022,Banyuwangi,IDN,East Java,89900


# SUB QUERY

In [18]:
# SubQUery sebagai kondisi
# Menampilkan negera dengan populasi diatas rata rata populasi dunia
# rata rata populasi dunia dipanggil dalam subquery

sql('''
SELECT name, population 
    FROM country 
    WHERE population>(SELECT AVG(population) FROM country) 
    ORDER BY population DESC;''').head()

,name,population
0,China,1277558000
1,India,1013662000
2,United States,278357000
3,Indonesia,212107000
4,Brazil,170115000


In [24]:
# SubQuery sebagai kolom baru

sql('''
SELECT name, population, (SELECT AVG(population) FROM country) AS rata_populasi
    FROM country
    LIMIT 5; 
''')

,name,population,rata_populasi
0,Aruba,103000,25434098.1172
1,Afghanistan,22720000,25434098.1172
2,Angola,12878000,25434098.1172
3,Anguilla,8000,25434098.1172
4,Albania,3401200,25434098.1172


# WORKING WITH MULTIPLE TABLE

In [25]:
sql('SELECT * FROM city LIMIT 1')

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000


In [26]:
sql('SELECT * FROM country LIMIT 1')

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.00,None,103000,78.4,828.00,793.00,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW


In [29]:
# nama negara dan kota kota nya dari negara yang regionnya adalah SE ASIA
# region, nama negara -> country
# nama kota dan populasi -> city

# CARA SUB QUERY
sql('''
SELECT countrycode, name, population 
    FROM city
    WHERE countrycode IN (SELECT code FROM country WHERE region='southeast asia')
    LIMIT 5
''')

,countrycode,name,population
0,BRN,Bandar Seri Begawan,21484
1,IDN,Jakarta,9604900
2,IDN,Surabaya,2663820
3,IDN,Bandung,2429000
4,IDN,Medan,1843919


In [31]:
# implicit join
sql('''
SELECT country.name AS negara, city.name AS kota, city.population AS populasi
    FROM country, city
    WHERE country.code = city.countrycode AND country.region = 'southeast asia'
''')

,negara,kota,populasi
0,Brunei,Bandar Seri Begawan,21484
1,Indonesia,Jakarta,9604900
2,Indonesia,Surabaya,2663820
3,Indonesia,Bandung,2429000
4,Indonesia,Medan,1843919
...,...,...,...
292,Vietnam,Cam Ranh,114041
293,Vietnam,Vinh,112455
294,Vietnam,My Tho,108404
295,Vietnam,Da Lat,106409


In [33]:
# INNER JOIN
sql('''
SELECT country.name AS negara, city.name AS kota, city.population AS populasi
    FROM country
    JOIN city
    ON country.code = city.countrycode 
    WHERE country.region = 'southeast asia'
    LIMIT 5
''')


,negara,kota,populasi
0,Brunei,Bandar Seri Begawan,21484
1,Indonesia,Jakarta,9604900
2,Indonesia,Surabaya,2663820
3,Indonesia,Bandung,2429000
4,Indonesia,Medan,1843919


In [34]:
sql('''
SELECT * FROM countrylanguage LIMIT 1
''')

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3


In [36]:
# Menampilkan negara dan bahasa resmi
sql('''
SELECT countrycode, COUNT(Language) AS jumlah_bahasa 
    FROM countrylanguage 
    WHERE IsOfficial='T' 
    GROUP BY countrycode 
    ORDER BY jumlah_bahasa DESC 
    LIMIT 5;
''')

,countrycode,jumlah_bahasa
0,CHE,4
1,ZAF,4
2,BOL,3
3,BEL,3
4,PER,3


In [38]:
# Menampilkan negara dan bahasa resmi
# country disingkat menjadi c dan countylanguage menjadi cl
sql('''
SELECT c.name AS negara, cl.language as bahasa
    FROM country c 
    JOIN countrylanguage cl
    ON cl.countrycode=c.code
    WHERE cl.isofficial='T'
''')

,negara,bahasa
0,Aruba,Dutch
1,Afghanistan,Dari
2,Afghanistan,Pashto
3,Anguilla,English
4,Albania,Albaniana
...,...,...
233,South Africa,Afrikaans
234,South Africa,English
235,South Africa,Xhosa
236,South Africa,Zulu


In [40]:
# TAMPILKAN NAMA NEGARA IBU KOTA NEGARA DAN BAHASA RESMI DARI NEGARA NEGARA DI SOUTH EAST ASIA
sql('SELECT * FROM CITY;').head()

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


In [42]:
sql('SELECT * FROM COUNTRY;').head()

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.00,NaN,103000,78.4,828.00,793.00,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.00,1919.0,22720000,45.9,5976.00,None,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.00,1975.0,12878000,38.3,6648.00,7984.00,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.00,NaN,8000,76.1,63.20,None,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.00,1912.0,3401200,71.6,3205.00,2500.00,Shqipëria,Republic,Rexhep Mejdani,34.0,AL


In [44]:
sql('SELECT * FROM COUNTRYLANGUAGE;').head()

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9


In [54]:
sql('''
SELECT country.name AS negara, city.name AS kota, countrylanguage.language AS bahasa
    FROM city
    JOIN country
    ON city.ID = country.capital
    JOIN countrylanguage
    ON country.code = countrylanguage.countrycode
    WHERE countrylanguage.isofficial='T' AND country.region='Southeast asia'
''')

,negara,kota,bahasa
0,Brunei,Bandar Seri Begawan,Malay
1,Indonesia,Jakarta,Malay
2,Cambodia,Phnom Penh,Khmer
3,Laos,Vientiane,Lao
4,Myanmar,Rangoon (Yangon),Burmese
5,Malaysia,Kuala Lumpur,Malay
6,Philippines,Manila,Pilipino
7,Singapore,Singapore,Chinese
8,Singapore,Singapore,Malay
9,Singapore,Singapore,Tamil


In [60]:
# LEFT JOIN
sql('''
SELECT country.name AS negara, city.name AS ibukota
    FROM city
    LEFT JOIN country
    ON city.id = country.capital
''')

,negara,ibukota
0,Afghanistan,Kabul
1,None,Qandahar
2,None,Herat
3,None,Mazar-e-Sharif
4,Netherlands,Amsterdam
...,...,...
4074,None,Khan Yunis
4075,None,Hebron
4076,None,Jabaliya
4077,None,Nablus
